# Multi-agent Session Notes

Different from the session manager of individual session, the final result is stored as well as metadata on which agent picked up the last task etc. 

If you try and give individual agents their own session when they are in a graph or other multi session, you will see the error `ValueError: Session persistence is not supported for Graph agents yet. `

In [10]:
from strands.multiagent import GraphBuilder
from strands.session.file_session_manager import FileSessionManager
from strands import Agent
from strands.models.ollama import OllamaModel
import pprint 


def needs_more_research(state):
    """Check if writer needs more research."""
    writer_result = state.results.get("writer")
    if not writer_result:
        return False
    result_text = str(writer_result.result).lower()
    return "need more research" in result_text

def writing_complete(state):
    """Check if writing is complete - this acts as the exit condition."""
    writer_result = state.results.get("writer")
    if not writer_result:
        return False
    result_text = str(writer_result.result).lower()
    return "writing complete" in result_text
    

model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.1",
)

# Multi-agent usage
multi_session_manager = FileSessionManager(
    session_id="orchestrator-456",
    storage_dir="./multi_agent_sessions"
)

# Create agents WITH system prompts that guide the cycle behavior
agent1 = Agent(
    model=model, 
    name="researcher",
    system_prompt="""Research the given topic thoroughly. 
    Provide comprehensive information based on what the writer needs.
    If this is follow-up research, focus on the specific areas requested."""
)

agent2 = Agent(
    model=model, 
    name="writer",
    system_prompt="""Write about the research provided.
    
    IMPORTANT: End your response with EXACTLY one of these phrases:
    - "NEED MORE RESEARCH ON: [specific topic]" - if you need additional information
    - "WRITING COMPLETE" - if you have everything needed to finish
    
    Be specific about what additional research you need."""
)

    
builder = GraphBuilder()
builder.add_node(agent1, "researcher")
builder.add_node(agent2, "writer")

# Create edges for the cycle
builder.add_edge("researcher", "writer")  # researcher outputs to writer
builder.add_edge("writer", "researcher", condition=needs_more_research)  # Loop back if more research needed

builder.set_entry_point("researcher")  # Start with researcher
builder.set_session_manager(multi_session_manager)

# Critical: Set execution limits to prevent infinite loops
builder.set_max_node_executions(10)  # Maximum 10 node executions total
builder.set_execution_timeout(4)  # 5 minute timeout
builder.reset_on_revisit(True)  # Reset node state when revisiting

graph = builder.build()

# Use the graph - all orchestrator state is persisted
result = graph("Research and write about AI")
print("\n=== Final Result ===")
print(f"Status: {result.status}")
print(f"Execution order: {[node.node_id for node in result.execution_order]}")
print(f"Total executions: {len(result.execution_order)}")

# Access results from specific nodes
print(f"\n=== Researcher Output ===")
if "researcher" in result.results:
    print(result.results["researcher"].result)

print(f"\n=== Writer Output ===")
if "writer" in result.results:
    print(result.results["writer"].result)

# Performance metrics
print(f"\n=== Metrics ===")
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

After reviewing the provided research on Artificial Intelligence (AI), it appears that this comprehensive overview covers various aspects of AI, including its history, types, applications, benefits, challenges, and future prospects. The researcher has provided a thorough understanding of AI's current state and potential.

 I realize that there are some areas where additional research could be beneficial to delve deeper into specific topics:

intelligence**: While the overview mentions superintelligence as one of the types of AI, it would be beneficial to explore this topic further, including its potential risks and benefits.
Bias and Inequality**: The researcher highlights the concern of bias in AI systems but does not provide detailed information on how to mitigate these issues. Additional research could focus on strategies for addressing bias and promoting fairness in AI development.
Human-AI Collaboration**: This topic is mentioned as a future prospect, but further investigation int

In [9]:
# we can also store state in S3 
from strands import Agent
from strands.session.s3_session_manager import S3SessionManager
import boto3

# # Optional: Create a custom boto3 session
# boto_session = boto3.Session(region_name="us-west-2")

# # Create a session manager that stores data in S3
# session_manager = S3SessionManager(
#     session_id="user-456",
#     bucket="my-agent-sessions",
#     prefix="production/",  # Optional key prefix
#     boto_session=boto_session,  # Optional boto3 session
#     region_name="us-west-2"  # Optional AWS region (if boto_session not provided)
# )